In [3]:
from alefuncs import *

In [40]:
from subprocess import run
import pexpect
from getpass import getpass

def ssl_encrypt(infile, outfile):
    '''(file_path, file_path) => encrypted_file
    Uses openssl to encrypt/decrypt files.
    '''
    pwd = getpass('enter encryption pwd:')
    if getpass('repeat pwd:') == pwd:
        run(f'openssl enc -aes-256-cbc -a -salt -pass pass:{pwd} -in {infile} -out {outfile}',shell=True)
    else:
        print("passwords don't match.")

    
def ssl_decrypt(infile, outfile):
    '''(file_path, file_path) => decrypted_file
    Uses openssl to encrypt/decrypt files.
    '''
    pwd = getpass('enter decryption pwd:')
    run(f'openssl enc -d -aes-256-cbc -a -pass pass:{pwd} -in {infile} -out {outfile}', shell=True)

In [25]:
infile = 'alefuncs.py'
outfile = 'alefuncs.enc'
print(f'openssl aes-256-cbc -a -salt -in {infile} -out {outfile} -base64')

infile = 'alefuncs.enc'
outfile = 'alefuncs.dec.py'
print(f'openssl enc -d -aes-256-cbc -in {infile} -out {outfile}')

openssl aes-256-cbc -a -salt -in alefuncs.py -out alefuncs.enc
openssl enc -d -aes-256-cbc -in alefuncs.enc -out alefuncs.dec.py


In [42]:
ssl_encrypt('alefuncs.py','alefuncs.enc')
ssl_decrypt('alefuncs.enc', 'a2.py')

enter encryption pwd:········
repeat pwd:········
enter decryption pwd:········


enter decryption pwd:········


In [33]:
run(f'openssl enc -aes-256-cbc -pass  -p -in alefuncs.py -out alefuncs.enc -base64',shell=True)


password········


CompletedProcess(args='openssl enc -aes-256-cbc -pass hayoo666 -p -in alefuncs.py -out alefuncs.enc -base64', returncode=1)